## Introduction

In this project we're going to practice writting more complex SQL queries to answer business questions.

We'll be using the [Chinook database]() for this project. The Chinook database is provided as a SQLite database file called chinook.db

## Loading the Database

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Overview of the Data

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Selecting New Albums to Purchase

Suppose the Chinook record store has just signed a deal with a new record label, and we've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

| Artist Name |  Genre  |         
|-------------|---------|
|Regal     | Hip-hop |
| Red Tone | Punk |
| Meteor and the Girls| Pop|

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

We'll need to write a query to find out which genres sell the most tracks in the USA  and make a recommendation for the three artists whose albums we should purchase for the store.

In [33]:
%%sql
WITH usa_tracks_sold AS
    (
        SELECT il.*
        FROM invoice_line il
        INNER JOIN invoice i ON i.invoice_id = il.invoice_id
        INNER JOIN customer c ON c.customer_id = i.customer_id
        WHERE c.country = 'USA'
    )
SELECT g.name genre, 
       COUNT(uts.invoice_line_id) tracks_sold,
       CAST(COUNT(uts.invoice_line_id) AS FLOAT)/
        (SELECT COUNT(*) from usa_tracks_sold) tracks_sold_percentage
FROM genre g
INNER JOIN track t ON t.genre_id = g.genre_id
INNER JOIN usa_tracks_sold uts ON uts.track_id = t.track_id
GROUP BY genre
ORDER BY tracks_sold DESC

Done.


genre,tracks_sold,tracks_sold_percentage
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


On a quick glance, In the US, the most popular music genre by far is Rock with the total of rock tracks sold more than double the sum of total tracks sold of runner up most popular genre (Alternative & Punk) and the third most popular genre (Metal) combine.

based on sales of tracks in the US we should pick up the musics of:

* Red Tone (Punk)
* Slim Jim Bites (Blues)
* Meteor and the Girls (Pop)

Due to the hip-hop genre is not very popular currently in the US

Alternatively, we can pick ups a few more Rock albums as the Rock genre is quite popular at the moment


## Analyzing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. 

We want to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [25]:
%%sql
WITH employee_sales_per AS
(
    SELECT e.first_name || ' ' || e.last_name employee,
           e.hire_date,
           e.title,
           e.employee_id, e.reports_to,
           i.total
    FROM employee e
    LEFT JOIN customer c ON c.support_rep_id = e.employee_id
    LEFT JOIN invoice i ON i.customer_id = c.customer_id
)
SELECT esp1.employee,
       esp1.hire_date 'hire date',
       esp1.title,    
       esp2.employee supervisor,
       ROUND(SUM(esp1.total),2) 'total sales (dollar)'
FROM employee_sales_per esp1
LEFT JOIN employee_sales_per esp2 ON esp1.reports_to = esp2.employee_id
GROUP BY 1
ORDER By 5 DESC



Done.


employee,hire date,title,supervisor,total sales (dollar)
Jane Peacock,2017-04-01 00:00:00,Sales Support Agent,Nancy Edwards,1731.51
Margaret Park,2017-05-03 00:00:00,Sales Support Agent,Nancy Edwards,1584.0
Steve Johnson,2017-10-17 00:00:00,Sales Support Agent,Nancy Edwards,1393.92
Andrew Adams,2016-08-14 00:00:00,General Manager,None,None
Laura Callahan,2017-03-04 00:00:00,IT Staff,Michael Mitchell,None
Michael Mitchell,2016-10-17 00:00:00,IT Manager,Andrew Adams,None
Nancy Edwards,2016-05-01 00:00:00,Sales Manager,Andrew Adams,None
Robert King,2017-01-02 00:00:00,IT Staff,Michael Mitchell,None


Jane Peacock has the most sales and is also the most senior sale support agent,
Steve Johnson has the least sales but he is also the newest member of the team. This result is some what expected as the difference roughly corresponds with the differences in their experience. 

## Analyzing Sales by Country

Next, we will analyze the sales data for customers from each different country. 

In particular, we want to calculate data, for each country, on the:

* total number of customers
* total value of sales
* average value of sales per customer
* average order value

In [52]:
%%sql
WITH country_classifier AS
(
 SELECT
       CASE
           WHEN (
                 SELECT count(DISTINCT customer_id)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
        )
SELECT country,
       total_number_of_customers,
       total_value_of_sales,
       average_value_of_sales_per_customer,
       average_order
FROM (
    SELECT country,
           COUNT(DISTINCT customer_id) total_number_of_customers,
           ROUND(SUM(unit_price), 3) total_value_of_sales,
           ROUND(CAST(SUM(unit_price) AS FLOAT)/COUNT(DISTINCT customer_id), 3) average_value_of_sales_per_customer,
           ROUND(CAST(SUM(unit_price) AS FLOAT)/count(distinct invoice_id), 3) average_order,
           CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_classifier
    GROUP BY country
    ORDER BY sort ASC, total_value_of_sales DESC
    )
       
       


Done.


country,total_number_of_customers,total_value_of_sales,average_value_of_sales_per_customer,average_order
USA,13,1040.49,80.038,7.943
Canada,8,535.59,66.949,7.047
Brazil,5,427.68,85.536,7.011
France,5,389.07,77.814,7.781
Germany,4,334.62,83.655,8.161
Czech Republic,2,273.24,136.62,9.108
United Kingdom,3,245.52,81.84,8.769
Portugal,2,185.13,92.565,6.384
India,2,183.15,91.575,8.721
Other,15,1094.94,72.996,7.449


Based on the data, there may be opportunity in the following countries:

* Czech Republic
* United Kingdom
* India

It's worth keeping in mind that because the amount of data from each of these countries is relatively low. Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.

## Albums vs Individual Tracks

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

* purchase a whole album
* purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We will find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [53]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases account for 18.6% of purchases. 

I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.